In [7]:
import os
import time
import pywt
import cv2
import json
import pickle
import itertools
import operator
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.io import loadmat
from tqdm import tqdm
from scipy import signal
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
# from stockwell import st
from scipy.signal import chirp

In [12]:
def most_common(L):
    # get an iterable of (item, iterable) pairs
    SL = sorted((x, i) for i, x in enumerate(L))
    # print 'SL:', SL
    groups = itertools.groupby(SL, key=operator.itemgetter(0))
    # auxiliary function to get "quality" for an item
    def _auxfun(g):
        item, iterable = g
        count = 0
        min_index = len(L)
        for _, where in iterable:
            count += 1
            min_index = min(min_index, where)
        # print 'item %r, count %r, minind %r' % (item, count, min_index)
        return count, -min_index
        # pick the highest-count/earliest item
    return max(groups, key=_auxfun)[0]

PHYSIONET

STFT

In [8]:
save_path = '/home/ishaan/Desktop/stft/'
json_path = '/home/ishaan/github/ecg/examples/cinc17/'
json_path_train = os.path.join(json_path,'train.json')
json_path_test = os.path.join(json_path,'dev.json')

if not os.path.exists(save_path):
    os.mkdir(save_path)

if not os.path.exists(os.path.join(save_path,'train')):
    os.mkdir(os.path.join(save_path,'train'))
os.mkdir(os.path.join(save_path,'train','ecg'))
if not os.path.exists(os.path.join(save_path,'val')):
    os.mkdir(os.path.join(save_path,'val'))
os.mkdir(os.path.join(save_path,'val','ecg'))

with open(json_path_train) as f:
    data_train = [json.loads(l) for l in f]
    
with open(json_path_test) as f:
    data_test = [json.loads(l) for l in f]

In [8]:
# s = loadmat(data_train[2]['ecg'])['val'][0,:]
# l = data_train[2]['labels']

In [5]:
# s.shape

(9000,)

In [9]:
# len(l)

30

In [10]:
fs = 300
STEP = 300
win_sz = 100;
overlap = 50;
win = signal.windows.hann(win_sz)
name = 0
js = {}
subject = 0
imgs = []
for line in tqdm(data_test):
    s = loadmat(line['ecg'])['val'][0,:]
    l = line['labels']
#     break
    length = s.shape[0]
    start = 0
    end = STEP
# #     for i in range(0,int(np.floor(length/STEP))):
    while end < length:
        
        w = s[start: end]
        st = time.time()
        f,t,zxx = signal.stft(s[start : end],fs, window=win, nperseg=win_sz, noverlap=overlap, nfft=win_sz, return_onesided=True, boundary='zeros', padded=True, axis=- 1)
        
        # plt.figure(figsize=(33,21))
        fig = Figure()
        fig.subplots_adjust(0,0,1,1)
        #         fig.add_axes([0,0,1,1])
        canvas = FigureCanvas(fig)
        ax = fig.gca()
        ax.pcolormesh(t, f, 20*np.log10(np.abs(zxx)), shading='gouraud')
        ax.axis('off')
        canvas.draw()
        width, height = fig.get_size_inches() * fig.get_dpi()
        image = np.frombuffer(canvas.tostring_rgb(), dtype='uint8').reshape(int(height), int(width), 3)
# #         if name%10 == 0:
        c = cv2.resize(image,(224,224))
        plt.imsave(os.path.join(save_path,'val','ecg',str(subject)+'_'+str(name)+'.jpg'),c)
        np.save(os.path.join(save_path,'val','ecg',str(subject)+'_'+str(name)+'.npy'),w)
        imgs.append(str(subject)+'_'+str(name))
        print(time.time()-st)
        name +=1
        if name > 9:
            js[str(subject)] = {'x': imgs, 'y': l[0]}
            imgs = []
            subject +=1
            name = 0
# #         else:
#         c = cv2.resize(image,(224,224))
#         plt.imsave(os.path.join(save_path,'train',l,str(name)+'.jpg'),c)
        
#         name += 1
        start += STEP
        end += STEP

with open(os.path.join(save_path,'val','ecg_labels.json'),'w') as f:
    json.dump(js,f,indent=2)

  0%|          | 1/852 [00:00<02:40,  5.29it/s]

0.01947808265686035
0.017448902130126953
0.03655219078063965
0.017522811889648438
0.02260446548461914
0.01672673225402832
0.024638652801513672
0.016927242279052734
0.016112804412841797
0.021486520767211914
0.018292903900146484
0.0321657657623291
0.02158808708190918
0.016816377639770508
0.01928091049194336
0.016826391220092773
0.016911029815673828
0.01692819595336914
0.02007150650024414
0.018824338912963867
0.0162661075592041
0.01725912094116211
0.016139984130859375
0.016547203063964844
0.01651310920715332
0.08883333206176758
0.017133712768554688
0.01666879653930664
0.01670098304748535


  0%|          | 2/852 [00:00<04:25,  3.20it/s]

0.016730308532714844
0.018219709396362305
0.016825199127197266
0.017253875732421875
0.016713857650756836
0.01674199104309082
0.018376827239990234
0.017276525497436523
0.017246723175048828
0.01656794548034668
0.01666879653930664
0.020305156707763672


  0%|          | 3/852 [00:01<04:11,  3.38it/s]

...

0.017147302627563477
0.019098758697509766
0.018762826919555664
0.016157865524291992
0.01641678810119629
0.016540050506591797
0.018863439559936523
0.018297672271728516
0.016784191131591797
0.016644716262817383
0.01705026626586914
0.019745349884033203
0.020795106887817383
0.01668858528137207
0.023706436157226562
0.018299102783203125
0.018574953079223633
0.016561508178710938
0.018711566925048828
0.016366958618164062
0.019526243209838867
0.01866769790649414
0.016206979751586914
0.01792168617248535
0.01977705955505371
0.018016815185546875
0.01676154136657715
0.016475677490234375
0.021856307983398438
0.01646709442138672
0.01621103286743164
0.016251564025878906
0.017073392868041992
0.03593111038208008


100%|█████████▉| 851/852 [09:07<00:00,  1.71it/s]

0.01874399185180664
0.016445636749267578
0.020179271697998047
0.016890287399291992
0.016958951950073242
0.019199848175048828
0.018772363662719727
0.017606258392333984
0.019954681396484375
0.017769813537597656
0.10107278823852539
0.021379470825195312
0.018146753311157227
0.016928672790527344
0.016636133193969727
0.01837635040283203
0.0163729190826416
0.01664113998413086
0.016605615615844727
0.01691126823425293
0.021263599395751953
0.017287492752075195


100%|██████████| 852/852 [09:08<00:00,  1.55it/s]

0.017621278762817383
0.017088651657104492
0.017079591751098633
0.018748998641967773
0.02000570297241211
0.017256975173950195
0.016291141510009766
0.016518115997314453
0.020323514938354492


In [3]:
save_path = '/home/vidhiwar/Desktop/stft/'
json_path = '/home/vidhiwar/github/ecg/examples/cinc17/'
json_path_train = os.path.join(json_path,'train.json')
json_path_test = os.path.join(json_path,'dev.json')

if not os.path.exists(os.path.join(save_path,'train')):
    os.mkdir(os.path.join(save_path,'train'))
if not os.path.exists(os.path.join(save_path,'val')):
    os.mkdir(os.path.join(save_path,'val'))



if not os.path.exists(os.path.join(save_path,'train','~')):
    os.mkdir(os.path.join(save_path,'train','~'))
if not os.path.exists(os.path.join(save_path,'train','A')):
    os.mkdir(os.path.join(save_path,'train','A'))
if not os.path.exists(os.path.join(save_path,'train','N')):
    os.mkdir(os.path.join(save_path,'train','N'))
if not os.path.exists(os.path.join(save_path,'train','O')):
    os.mkdir(os.path.join(save_path,'train','O'))
    
if not os.path.exists(os.path.join(save_path,'val','~')):
    os.mkdir(os.path.join(save_path,'val','~'))
if not os.path.exists(os.path.join(save_path,'val','A')):
    os.mkdir(os.path.join(save_path,'val','A'))
if not os.path.exists(os.path.join(save_path,'val','N')):
    os.mkdir(os.path.join(save_path,'val','N'))
if not os.path.exists(os.path.join(save_path,'val','O')):
    os.mkdir(os.path.join(save_path,'val','O'))
    
with open(json_path_train) as f:
    data_train = [json.loads(l) for l in f]
    
with open(json_path_test) as f:
    data_test = [json.loads(l) for l in f]

In [5]:
fs = 300
STEP = 3000
win_sz = 512;
overlap = 256;
win = signal.windows.hann(win_sz)
name = 0
for line in tqdm(data_train):
    s = loadmat(line['ecg'])['val'][0,:]
    l = line['labels'][0]
#     break
    length = s.shape[0]
    start = 0
    end = STEP
#     for i in range(0,int(np.floor(length/STEP))):
    while end < length:
        f,t,zxx = signal.stft(s[start : end],fs, window=win, nperseg=win_sz, noverlap=overlap, nfft=win_sz, return_onesided=True, boundary='zeros', padded=True, axis=- 1)
        # plt.figure(figsize=(33,21))
        fig = Figure()
        fig.subplots_adjust(0,0,1,1)
        #         fig.add_axes([0,0,1,1])
        canvas = FigureCanvas(fig)
        ax = fig.gca()
        ax.pcolormesh(t, f, 20*np.log10(np.abs(zxx)), shading='gouraud')
        ax.axis('off')
        canvas.draw()
        width, height = fig.get_size_inches() * fig.get_dpi()
        image = np.frombuffer(canvas.tostring_rgb(), dtype='uint8').reshape(int(height), int(width), 3)
#         if name%10 == 0:
#             c = cv2.resize(image,(224,224))
#             plt.imsave(os.path.join(save_path,'val',l,str(name)+'.jpg'),c)
#         else:
        c = cv2.resize(image,(224,224))
        plt.imsave(os.path.join(save_path,'train',l,str(name)+'.jpg'),c)
        
        name += 1
        start += 1500
        end += 1500
conda install cv2

100%|██████████| 7676/7676 [19:27<00:00,  6.57it/s]


In [4]:
fs = 300
STEP = 3000
win_sz = 512
overlap = 256
win = signal.windows.hann(win_sz)
name = 0
for line in tqdm(data_test):
    s = loadmat(line['ecg'])['val'][0,:]
    l = line['labels'][0]
#     break
    length = s.shape[0]
    start = 0
    end = STEP
#     for i in range(0,int(np.floor(length/STEP))):
    while end < length:
        f,t,zxx = signal.stft(s[start : end],fs, window=win, nperseg=win_sz, noverlap=overlap, nfft=win_sz, return_onesided=True, boundary='zeros', padded=True, axis=- 1)
        # plt.figure(figsize=(33,21))
        fig = Figure()
        fig.subplots_adjust(0,0,1,1)
        #         fig.add_axes([0,0,1,1])
        canvas = FigureCanvas(fig)
        ax = fig.gca()
        ax.pcolormesh(t, f, 20*np.log10(np.abs(zxx)), shading='gouraud')
        ax.axis('off')
        canvas.draw()
        width, height = fig.get_size_inches() * fig.get_dpi()
        image = np.frombuffer(canvas.tostring_rgb(), dtype='uint8').reshape(int(height), int(width), 3)
#         if name%10 == 0:
#             c = cv2.resize(image,(224,224))
#             plt.imsave(os.path.join(save_path,'val',l,str(name)+'.jpg'),c)
#         else:
        c = cv2.resize(image,(224,224))
        plt.imsave(os.path.join(save_path,'val',l,str(name)+'.jpg'),c)
        
        name += 1
        start += 1500
        end += 1500

100%|██████████| 852/852 [02:10<00:00,  6.55it/s]


CWT

In [3]:
save_path = '/home/vidhiwar/Desktop/cwt/'
json_path = '/home/vidhiwar/github/ecg/examples/cinc17/'
json_path_train = os.path.join(json_path,'train.json')
json_path_test = os.path.join(json_path,'dev.json')

if not os.path.exists(os.path.join(save_path,'train')):
    os.mkdir(os.path.join(save_path,'train'))
if not os.path.exists(os.path.join(save_path,'val')):
    os.mkdir(os.path.join(save_path,'val'))


if not os.path.exists(os.path.join(save_path,'train','~')):
    os.mkdir(os.path.join(save_path,'train','~'))
if not os.path.exists(os.path.join(save_path,'train','A')):
    os.mkdir(os.path.join(save_path,'train','A'))
if not os.path.exists(os.path.join(save_path,'train','N')):
    os.mkdir(os.path.join(save_path,'train','N'))
if not os.path.exists(os.path.join(save_path,'train','O')):
    os.mkdir(os.path.join(save_path,'train','O'))
    
if not os.path.exists(os.path.join(save_path,'val','~')):
    os.mkdir(os.path.join(save_path,'val','~'))
if not os.path.exists(os.path.join(save_path,'val','A')):
    os.mkdir(os.path.join(save_path,'val','A'))
if not os.path.exists(os.path.join(save_path,'val','N')):
    os.mkdir(os.path.join(save_path,'val','N'))
if not os.path.exists(os.path.join(save_path,'val','O')):
    os.mkdir(os.path.join(save_path,'val','O'))
    
with open(json_path_train) as f:
    data_train = [json.loads(l) for l in f]
    
with open(json_path_test) as f:
    data_test = [json.loads(l) for l in f]

In [6]:
n = 0
widths = np.arange(1, 512)

for line in tqdm(data_train):
    s = loadmat(line['ecg'])['val']
    l = line['labels']
    start = 0
    end = 3000
    while end < s.shape[1]:
        c = signal.cwt(s[0,start:end],signal.ricker,widths)


        plt.imsave(os.path.join(save_path,'train',l[0],str(n)+'.jpg'),c)
        c = plt.imread(os.path.join(save_path,'train',l[0],str(n)+'.jpg'))
        c = cv2.resize(c,(224,224))
        plt.imsave(os.path.join(save_path,'train',l[0],str(n)+'.jpg'),c)
        
        start += 1500
        end += 1500
        n += 1
    
    

100%|██████████| 7676/7676 [2:08:28<00:00,  1.00s/it]  


In [5]:
n = 0
widths = np.arange(1, 512)

for line in tqdm(data_test):
    s = loadmat(line['ecg'])['val']
    l = line['labels']
    start = 0
    end = 3000
    while end < s.shape[1]:
        c = signal.cwt(s[0,start:end],signal.ricker,widths)

        plt.imsave(os.path.join(save_path,'val',l[0],str(n)+'.jpg'),c)
        c = plt.imread(os.path.join(save_path,'val',l[0],str(n)+'.jpg'))
        c = cv2.resize(c,(224,224))
        plt.imsave(os.path.join(save_path,'val',l[0],str(n)+'.jpg'),c)

        start += 1500
        end += 1500
        n += 1

100%|██████████| 852/852 [14:17<00:00,  1.01s/it]


MIT-BIH

STFT

In [ ]:
df = pd.read_excel('/home/vidhiwar/Desktop/ECG_Data_Collect.xlsx')

In [ ]:
fs = 360
STEP = 3600
win_sz = 512;
overlap = 256;
zero = 1024
g_dict = {'M':[0,1],'F':[1,0]}
save_path = '/home/vidhiwar/Desktop/stft/'

l2n = {'N':0, 'L':1, 'R':2, 'V':3, 'A':4,'~':5}

train = []
val = []

win = signal.windows.hann(win_sz)
name = 0
for j in tqdm(range(0,48,1)):
    i = df['#record'][j]
    if os.path.exists('/home/vidhiwar/github/ecg/examples/mitdb/mit/' + str(i) + '.pkl'):
        with open('/home/vidhiwar/github/ecg/examples/mitdb/mit/' + str(i) + '.pkl', 'rb') as f:
            data = pickle.load(f)
        w = np.load('/home/vidhiwar/github/ecg/examples/mitdb/mit/' + str(i) + '.npy')
        l_dict = {}
        for a in data:
            if a[2] in ['N', 'L', 'R', 'V', 'A']:
                l_dict[a[1]] = a[2]
            else:
                l_dict[a[1]] = '~'

        s = w[:,2]
        g = g_dict[df['Sex'][j]]
        age = 0
        if df['Age'][j] > 0 and df['Age'][j] < 100:
            age = df['Age'][j]/100
            
        
        #    break
        length = s.shape[0]
        start = 0
        end = STEP
        #     for i in range(0,int(np.floor(length/STEP))):
        while end < length:
            sample = {}
            f,t,zxx = signal.stft(s[start : end]-zero,fs, window=win, nperseg=win_sz, noverlap=overlap, nfft=win_sz, return_onesided=True, boundary='zeros', padded=True, axis=- 1)
                # plt.figure(figsize=(33,21))
            fig = Figure()
            fig.subplots_adjust(0,0,1,1)
                #         fig.add_axes([0,0,1,1])
            canvas = FigureCanvas(fig)
            ax = fig.gca()
            ax.pcolormesh(t, f, 20*np.log10(np.abs(zxx)), shading='gouraud')
            ax.axis('off')
            canvas.draw()
            width, height = fig.get_size_inches() * fig.get_dpi()
            image = np.frombuffer(canvas.tostring_rgb(), dtype='uint8').reshape(int(height), int(width), 3)
#             key = list(l_dict.keys())[0]
            label_list = []
            for k in range(start,end):
                if k in list(l_dict.keys()):conda install cv2
                    label_list.append(l_dict[k])
            
            l = most_common(label_list)
            
            sample['image'] = str(name)+'.jpg'
            sample['id'] = name
            sample['label'] = l2n[l]
            sample['age'] = age
            sample['gender'] = g
            if name%10 == 0: conda install -c conda-forge opencv 
                c = cv2.resize(image,(224,224))
                plt.imsave(os.path.join(save_path,'val','images',str(name)+'.jpg'),c)
                val.append(sample)
            else:
                c = cv2.resize(image,(224,224))
                plt.imsave(os.path.join(save_path,'train','images',str(name)+'.jpg'),c)
                train.append(sample)

            name += 1
            start += 1800
            end += 1800
ecg_train = {}
ecg_val = {}


ecg_train['data'] = train
ecg_val['data'] = val

with open('/home/vidhiwar/Desktop/stft/train/ecg_labels.json','w') as f:
    json.dump(ecg_train,f,indent=2)

with open('/home/vidhiwar/Desktop/stft/val/ecg_labels.json','w') as f:
    json.dump(ecg_val,f,indent=2)

CWT

In [3]:
df = pd.read_excel('/home/vidhiwar/Desktop/ecg/ECG_Data_Collect.xlsx')

In [ ]:
STEP = 3600

zero = 1024
g_dict = {'M':[0,1],'F':[1,0]}
save_path = '/home/vidhiwar/Desktop/cwt/'
df = pd.read_excel('/home/vidhiwar/Desktop/ECG_Data_Collect.xlsx')
l2n = {'N':0, 'L':1, 'R':2, 'V':3, 'A':4,'~':5}
widths = np.arange(1, 512)
train = []
val = []
name = 0
for j in tqdm(range(0,48,1)):
    i = df['#record'][j]
    if os.path.exists('/home/vidhiwar/github/ecg/examples/mitdb/mit/' + str(i) + '.pkl'):
        with open('/home/vidhiwar/github/ecg/examples/mitdb/mit/' + str(i) + '.pkl', 'rb') as f:
            data = pickle.load(f)
        w = np.load('/home/vidhiwar/github/ecg/examples/mitdb/mit/' + str(i) + '.npy')
        l_dict = {}
        for a in data:
            if a[2] in ['N', 'L', 'R', 'V', 'A']:
                l_dict[a[1]] = a[2]
            else:
                l_dict[a[1]] = '~'

        s = w[:,2]
        g = g_dict[df['Sex'][j]]
        age = 0conda install cv2
        if df['Age'][j] > 0 and df['Age'][j] < 100:
            age = df['Age'][j]/100
            
        
        #    break
        length = s.shape[0]
        start = 0
        end = STEP
        #     for i in range(0,int(np.floor(length/STEP))):
        while end < length:
            sample = {}
            c = signal.cwt(s[start:end]-zero,signal.ricker,widths)
#             key = list(l_dict.keys())[0]
            label_list = []
            for k in range(start,end):
                if k in list(l_dict.keys()):
                    label_list.append(l_dict[k])
            conda install cv2
            l = most_common(label_list)
            
            sample['image'] = str(name)+'.jpg'/home/ishaan/data/ecg/res/mitdb/mit
            sample['id'] = name
            sample['label'] = l2n[l]
            sample['age'] = age
            sample['gender'] = g
            if name%10 == 0:
                plt.imsave(os.path.join(save_path,'val','images',str(name)+'.jpg'),c)
                c = plt.imread(os.path.join(save_path,'val','images',str(name)+'.jpg'))
                c = cv2.resize(c,(224,224))
                plt.imsave(os.path.join(save_path,'val','images',str(name)+'.jpg'),c)
                val.append(sample)
            else:
                plt.imsave(os.path.join(save_path,'train','images',str(name)+'.jpg'),c)
                c = plt.imread(os.path.join(save_path,'train','images',str(name)+'.jpg'))
                c = cv2.resize(c,(224,224))
                plt.imsave(os.path.join(save_path,'train','images',str(name)+'.jpg'),c)
                train.append(sample)

            name += 1
            start += 1800
            end += 1800
ecg_train = {}
ecg_val = {}


ecg_train['data'] = train
ecg_val['data'] = val
conda install cv2
with open('/home/vidhiwar/Desktop/cwt/train/ecg_labels.json','w') as f:
    json.dump(ecg_train,f,indent=2)

with open('/home/vidhiwar/Desktop/cwt/val/ecg_labels.json','w') as f:
    json.dump(ecg_val,f,indent=2)

MITBIH

STFT, AGE, GENDER, RAW

In [13]:
save_path = '/home/vidhiwar/Desktop/ecg/'

mitbih_path = '/home/vidhiwar/github/ecg/examples/mitdb/mit/'

df = pd.read_excel('/home/vidhiwar/work_dir/ecg_spectogram/phase1/ecg_res/ECG_Data_Collect.xlsx')

if not os.path.exists(os.path.join(save_path,'mitbih_rl')):
    os.mkdir(os.path.join(save_path,'mitbih_rl'))

save_path = os.path.join(save_path, 'mitbih_rl')

# for i in ['stft','cwt']:


#     if not os.path.exists(os.path.join(save_path,'img_folder')):
#         os.mkdir(os.path.join(save_path,'img_folder'))

#     save_path_img = os.path.join(save_path,'img_folder',i)

#     if not os.path.exists(os.path.join(save_path_img,'train')):
#         os.mkdir(os.path.join(save_path_img,'train'))
#     if not os.path.exists(os.path.join(save_path_img,'val')):
#         os.mkdir(os.path.join(save_path_img,'val'))

#     if not os.path.exists(os.path.join(save_path_img,'train','~')):
#         os.mkdir(os.path.join(save_path_img,'train','~'))
#     if not os.path.exists(os.path.join(save_path_img,'train','A')):
#         os.mkdir(os.path.join(save_path_img,'train','A'))
#     if not os.path.exists(os.path.join(save_path_img,'train','N')):
#         os.mkdir(os.path.join(save_path_img,'train','N'))
#     if not os.path.exists(os.path.join(save_path_img,'train','L')):
#         os.mkdir(os.path.join(save_path_img,'train','L'))
#     if not os.path.exists(os.path.join(save_path_img,'train','R')):
#         os.mkdir(os.path.join(save_path_img,'train','R'))
#     if not os.path.exists(os.path.join(save_path_img,'train','V')):
#         os.mkdir(os.path.join(save_path_img,'train','V'))

#     if not os.path.exists(os.path.join(save_path_img,'val','~')):
#         os.mkdir(os.path.join(save_path_img,'val','~'))
#     if not os.path.exists(os.path.join(save_path_img,'val','A')):
#         os.mkdir(os.path.join(save_path_img,'val','A'))
#     if not os.path.exists(os.path.join(save_path_img,'val','N')):
#         os.mkdir(os.path.join(save_path_img,'val','N'))
#     if not os.path.exists(os.path.join(save_path_img,'val','L')):
#         os.mkdir(os.path.join(save_path_img,'val','L'))
#     if not os.path.exists(os.path.join(save_path_img,'val','R')):
#         os.mkdir(os.path.join(save_path_img,'val','R'))
#     if not os.path.exists(os.path.join(save_path_img,'val','V')):
#         os.mkdir(os.path.join(save_path_img,'val','V'))

if not os.path.exists(os.path.join(save_path,'train')):
    os.mkdir(os.path.join(save_path,'train'))
if not os.path.exists(os.path.join(save_path,'val')):
    os.mkdir(os.path.join(save_path,'val'))
    
if not os.path.exists(os.path.join(save_path,'train','images')):
    os.mkdir(os.path.join(save_path,'train','images'))
if not os.path.exists(os.path.join(save_path,'val','images')):
    os.mkdir(os.path.join(save_path,'val','images'))
    
if not os.path.exists(os.path.join(save_path,'train','images_full')):
    os.mkdir(os.path.join(save_path,'train','images_full'))
if not os.path.exists(os.path.join(save_path,'val','images_full')):
    os.mkdir(os.path.join(save_path,'val','images_full'))

for i in ['stft','cwt']:
    if not os.path.exists(os.path.join(save_path,'train','images',i)):
        os.mkdir(os.path.join(save_path,'train','images',i))
    if not os.path.exists(os.path.join(save_path,'val','images',i)):
        os.mkdir(os.path.join(save_path,'val','images',i))
        
    if not os.path.exists(os.path.join(save_path,'train','images_full',i)):
        os.mkdir(os.path.join(save_path,'train','images_full',i))
    if not os.path.exists(os.path.join(save_path,'val','images_full',i)):
        os.mkdir(os.path.join(save_path,'val','images_full',i))
    
if not os.path.exists(os.path.join(save_path,'train','ecg')):
    os.mkdir(os.path.join(save_path,'train','ecg'))
if not os.path.exists(os.path.join(save_path,'val','ecg')):
    os.mkdir(os.path.join(save_path,'val','ecg'))

In [14]:
fs = 360
STEP = 3600
win_sz = 72
overlap = 36

outer_win_sz = 180
outer_overlap = 90

zero = 1024
inner_step = 360
widths = np.concatenate([np.flip(np.arange(1, 181)),np.arange(1, 181)])
outer_widths = np.concatenate([np.flip(np.arange(1, 721)),np.arange(1, 721)])
g_dict = {'M':[0,1],'F':[1,0]}

l2n = {'N':0, 'L':1, 'R':2, 'V':3, 'A':4,'~':5}

n2l = {0:'N', 1:'L', 2:'R', 3:'V', 4:'A',5:'~'}

train = []
val = []

win = signal.windows.hann(win_sz)
outer_win = signal.windows.hann(outer_win_sz)
name = 0
l_defaullt = '~'
for j in tqdm(range(0,2,1)):
    i = df['#record'][j]
    if os.path.exists(os.path.join(mitbih_path,str(i) + '.pkl')):
        with open(os.path.join(mitbih_path,str(i) + '.pkl'), 'rb') as f:
            data = pickle.load(f)
        w = np.load(os.path.join(mitbih_path,str(i) + '.npy'))
        l_dict = {}
        for a in data:
            if a[2] in ['N', 'L', 'R', 'V', 'A']:
                l_dict[a[1]] = a[2]
            else:
                l_dict[a[1]] = '~'

        s = w[:,2]
        g = g_dict[df['Sex'][j]]
        age = 0
        if df['Age'][j] > 0 and df['Age'][j] < 100:
            age = df['Age'][j]/100
            
        
        #    break
        length = s.shape[0]
        start = 0
        end = STEP
        #     for i in range(0,int(np.floor(length/STEP))):
        while end < length:

            sample = {}
            
            inner_start = start
            inner_end = start+inner_step
            
            segment_images = []
            segment_ecg = []
            segment_id = 0
            f,t,zxx = signal.stft(s[start : end]-zero,fs, window=outer_win, nperseg=outer_win_sz, noverlap=outer_overlap, nfft=outer_win_sz, return_onesided=True, boundary='zeros', padded=True, axis=- 1)
                # plt.figure(figsize=(33,21))
            fig = Figure()
            fig.subplots_adjust(0,0,1,1)
                #         fig.add_axes([0,0,1,1])
            canvas = FigureCanvas(fig)
            ax = fig.gca()
            ax.pcolormesh(t, f, 20*np.log10(np.abs(zxx)), shading='gouraud')
            ax.axis('off')
            canvas.draw()
            width, height = fig.get_size_inches() * fig.get_dpi()
            image = np.frombuffer(canvas.tostring_rgb(), dtype='uint8').reshape(int(height), int(width), 3)
            
            cwt = signal.cwt(s[start:end]-zero,signal.ricker,outer_widths)
            labels = []
            
            if name%10 == 0:
                plt.imsave(os.path.join(save_path,'val','images_full','cwt',str(name)+'.jpg'),cwt)
                cwt = plt.imread(os.path.join(save_path,'val','images_full','cwt',str(name)+'.jpg'))
                cwt = cv2.resize(cwt,(224,224))
                plt.imsave(os.path.join(save_path,'val','images_full','cwt',str(name)+'.jpg'),cwt)
                c = cv2.resize(image,(224,224))
                plt.imsave(os.path.join(save_path,'val','images_full','stft',str(name)+'.jpg'),c)
                
            else:
                plt.imsave(os.path.join(save_path,'train','images_full','cwt',str(name)+'.jpg'),cwt)
                cwt = plt.imread(os.path.join(save_path,'train','images_full','cwt',str(name)+'.jpg'))
                cwt = cv2.resize(cwt,(224,224))
                plt.imsave(os.path.join(save_path,'train','images_full','cwt',str(name)+'.jpg'),cwt)
                c = cv2.resize(image,(224,224))
                plt.imsave(os.path.join(save_path,'train','images_full','stft',str(name)+'.jpg'),c)
            while inner_end <= end:
            
                f,t,zxx = signal.stft(s[inner_start : inner_end]-zero,fs, window=win, nperseg=win_sz, noverlap=overlap, nfft=win_sz, return_onesided=True, boundary='zeros', padded=True, axis=- 1)
                    # plt.figure(figsize=(33,21))
                fig = Figure()
                fig.subplots_adjust(0,0,1,1)
                    #         fig.add_axes([0,0,1,1])
                canvas = FigureCanvas(fig)
                ax = fig.gca()
                ax.pcolormesh(t, f, 20*np.log10(np.abs(zxx)), shading='gouraud')
                ax.axis('off')
                canvas.draw()
                width, height = fig.get_size_inches() * fig.get_dpi()
                image = np.frombuffer(canvas.tostring_rgb(), dtype='uint8').reshape(int(height), int(width), 3)
                
                cwt = signal.cwt(s[inner_start:inner_end]-zero,signal.ricker,outer_widths)
                segment_ecg.append(str(name)+'_'+str(segment_id)+'.npy')
                segment_images.append(str(name)+'_'+str(segment_id)+'.jpg')
                if name%10 == 0:
                    plt.imsave(os.path.join(save_path,'val','images','cwt',str(name)+'_'+str(segment_id)+'.jpg'),cwt)
                    cwt = plt.imread(os.path.join(save_path,'val','images','cwt',str(name)+'_'+str(segment_id)+'.jpg'))
                    cwt = cv2.resize(cwt,(224,224))
                    plt.imsave(os.path.join(save_path,'val','images','cwt',str(name)+'_'+str(segment_id)+'.jpg'),cwt)
                    c = cv2.resize(image,(224,224))
                    plt.imsave(os.path.join(save_path,'val','images','stft',str(name)+'_'+str(segment_id)+'.jpg'),c)
                    np.save(os.path.join(save_path,'val','ecg',str(name)+'_'+str(segment_id)+'.npy'),s[inner_start : inner_end]-zero)
                else:
                    plt.imsave(os.path.join(save_path,'train','images','cwt',str(name)+'_'+str(segment_id)+'.jpg'),cwt)
                    cwt = plt.imread(os.path.join(save_path,'train','images','cwt',str(name)+'_'+str(segment_id)+'.jpg'))
                    cwt = cv2.resize(cwt,(224,224))
                    plt.imsave(os.path.join(save_path,'train','images','cwt',str(name)+'_'+str(segment_id)+'.jpg'),cwt)
                    c = cv2.resize(image,(224,224))
                    plt.imsave(os.path.join(save_path,'train','images','stft',str(name)+'_'+str(segment_id)+'.jpg'),c)
                    np.save(os.path.join(save_path,'train','ecg',str(name)+'_'+str(segment_id)+'.npy'),s[inner_start : inner_end]-zero)
                label_list = []
                for k in range(inner_start,inner_end):
                    if k in list(l_dict.keys()):
                        label_list.append(l_dict[k])
                
                if len(label_list) == 0:
                    l = default_label
                else:
                    l = most_common(label_list)
                    default_label = l
                labels.append(l2n[l])
                segment_id += 1
                inner_start += inner_step
                inner_end += inner_step
#             key = list(l_dict.keys())[0]
#             label_list = []
#             for k in range(start,end):
#                 if k in list(l_dict.keys()):
#                     label_list.append(l_dict[k])
            
#             l = most_common(label_list)
            
            sample['images'] = segment_images
            sample['images_full'] = str(name)+'.jpg'
            sample['ecg'] = segment_ecg
            sample['id'] = name
            sample['label'] = labels
            sample['age'] = age
            sample['gender'] = g
            if name%10 == 0:
                val.append(sample)
            else:
                train.append(sample)

            name += 1
            start += STEP
            end += STEP
ecg_train = {}
ecg_val = {}


ecg_train['data'] = train
ecg_val['data'] = val
ecg_train['labels'] = n2l
ecg_val['labels'] = n2l


with open(os.path.join(save_path,'train','ecg_labels.json'),'w') as f:
    json.dump(ecg_train,f,indent=2)

with open(os.path.join(save_path,'val','ecg_labels.json'),'w') as f:
    json.dump(ecg_val,f,indent=2)

100%|██████████| 2/2 [13:19<00:00, 399.85s/it]
